In [36]:
import numpy as np
import pandas as pd
import scipy as sp
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

## 가우시안 정규분포 나이브 베이즈 모형

In [2]:
np.random.seed(0)
rv0 = sp.stats.multivariate_normal([-2, -2], [[1, 0.9], [0.9, 2]])
rv1 = sp.stats.multivariate_normal([2, 2], [[1.2, -0.8], [-0.8, 2]])
X0 = rv0.rvs(40)
X1 = rv1.rvs(60)

In [3]:
X = np.vstack([X0, X1])
y = np.hstack([np.zeros(40), np.ones(60)])

In [4]:
from sklearn.naive_bayes import GaussianNB
model_norm = GaussianNB().fit(X, y)

In [11]:
print(model_norm.theta_[0], model_norm.sigma_[0])
print(model_norm.theta_[1], model_norm.sigma_[1])

[-1.96197643 -2.00597903] [1.02398854 2.31390497]
[2.19130701 2.12626716] [1.25429371 1.93742544]


In [15]:
x_new = [-0.7, -0.8]
model_norm.predict_proba([x_new])

array([[0.98300323, 0.01699677]])

In [17]:
likelihood = [
    sp.stats.norm(model_norm.theta_[0], np.sqrt(model_norm.sigma_[0])).pdf(x_new).prod(),
    sp.stats.norm(model_norm.theta_[1], np.sqrt(model_norm.sigma_[1])).pdf(x_new).prod(),
]
likelihood

[0.0346968402917242, 0.0003999540815189114]

In [18]:
model_norm.class_prior_

array([0.4, 0.6])

In [19]:
posterior = likelihood * model_norm.class_prior_
posterior

array([0.01387874, 0.00023997])

In [20]:
posterior / posterior.sum()

array([0.98300323, 0.01699677])

In [ ]:
# 연습문제 1 : 붓꽃 분류 문제를 가우시안 나이브 베이즈 모형을 사용하여 풀고 다음을 구하라.
# confusion matrix, classification report

In [35]:
from sklearn.naive_bayes import GaussianNB

In [22]:
from sklearn.datasets import load_iris
iris = load_iris()
df = pd.DataFrame(iris.data, columns=iris.feature_names)
sy = pd.Series(iris.target, dtype="category")
sy = sy.cat.rename_categories(iris.target_names)
df['species'] = sy
df.tail()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),species
145,6.7,3.0,5.2,2.3,virginica
146,6.3,2.5,5.0,1.9,virginica
147,6.5,3.0,5.2,2.0,virginica
148,6.2,3.4,5.4,2.3,virginica
149,5.9,3.0,5.1,1.8,virginica


In [23]:
X = df.iloc[:, :4]
y = df.iloc[:, 4:]

In [25]:
y.tail()

,species
145,virginica
146,virginica
147,virginica
148,virginica
149,virginica


In [28]:
model_iris = GaussianNB().fit(X, y)
model_iris

/usr/local/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GaussianNB(priors=None)

In [30]:
y_pred = model_iris.predict(X)
y_true = y.copy()

In [33]:
print(confusion_matrix(y_true, y_pred))

[[50  0  0]
 [ 0 47  3]
 [ 0  3 47]]


In [34]:
print(classification_report(y_true, y_pred))

             precision    recall  f1-score   support

     setosa       1.00      1.00      1.00        50
 versicolor       0.94      0.94      0.94        50
  virginica       0.94      0.94      0.94        50

avg / total       0.96      0.96      0.96       150



In [ ]:
# 연습문제 2 : MNIST Digit 분류 문제에서 sklearn.preprocessing.Binarizer로 x값을 0, 1로 바꾼다(값이 8 이상이면 1, 8 미만이면 0). 
# 즉 흰색과 검은색 픽셀로만 구성된 이미지로 만든다. 이 이미지에 대해 베르누이 나이브 베이즈 모형을 적용한다.

In [ ]:
from sklearn.preprocessing import Binarizer
binarizer = Binarizer(threshold=8)

In [ ]:
from sklearn.datasets import load_digits
X = load_digits().data
y = load_digits().target

In [ ]:
binarizer.fit_transform(X)

In [ ]:
from sklearn.naive_bayes import BernoulliNB

In [ ]:
model_bern = BernoulliNB().fit(X, y)

In [ ]:
y_pred = model_bern.predict(X)
y_true = y

In [ ]:
print(confusion_matrix(y_true, y_pred))
print("-" * 50)
print(classification_report(y_true, y_pred))

In [ ]:
theta = model_bern.feature_log_prob_
theta[0]

In [ ]:
plt.imshow(theta[3].reshape(8, 8))

In [ ]:
# 연습문제 3

In [ ]:
from sklearn.datasets import load_digits
X = load_digits().data
y = load_digits().target

In [ ]:
from sklearn.naive_bayes import MultinomialNB
model = MultinomialNB().fit(X, y)

In [ ]:
y_pred = model.predict(X)
y_true = y

In [ ]:
print(confusion_matrix(y_true, y_pred))
print("-" * 50)
print(classification_report(y_true, y_pred))

In [ ]:
theta = model.feature_log_prob_
theta[0]

In [ ]:
plt.imshow(theta[3].reshape(8, 8))